In [143]:
!pip install pydot graphviz

In [145]:
# Importing packages
import os
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import Model, layers
from tensorflow.keras.datasets import mnist
from keras.utils import plot_model
import pydot
import graphviz
print("pydot:", pydot.__version__)
print("graphviz:", graphviz.__version__)

pydot: 4.0.1
graphviz: 0.20.3


### Subclassing

In [14]:
class car():
    def __init__(self,model,colour):
        self.model=model
        self.colour=colour

    def show(self):
        print(f"The model of the car is: {self.model}")
        print(f"The colour of the car is: {self.colour}")

In [16]:
audi = car("audi r8","red")
mustang = car("mustang","yellow")

In [18]:
audi.show()

The model of the car is: audi r8
The colour of the car is: red


In [22]:
mustang.show()

The model of the car is: mustang
The colour of the car is: yellow


In [24]:
class Rectange:
    def __init__(self,length,width):
        self.length =length
        self.width =width

    def area(self):
        return self.length * self.width

class Square:
    def __init__(self,length):
        self.length =length

    def area(self):
        return self.length * self.length

In [26]:
square = Square(4)
square.area()

16

In [28]:
rectange = Rectange(4,5)
rectange.area()

20

#### Inheritance

In [34]:
class Rectange:
    def __init__(self,length,width):
        self.length =length
        self.width =width

    def area(self):
        return self.length * self.width

class Square(Rectange):
     def __init__(self,length):
        super().__init__(length,length)


In [36]:
square = Square(4)
square.area()

16

In [73]:
(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train = x_train.reshape(-1, 28 * 28).astype("float32") / 255.0
x_test = x_test.reshape(-1, 28 * 28).astype("float32") / 255.0

In [75]:
x_train.shape,x_test.shape

((60000, 784), (10000, 784))

In [111]:
class MyModel(keras.Model):
    def __init__(self, num_classes=10):
        super().__init__()
        self.dense1 = layers.Dense(64, activation='relu')  
        self.dense2 = layers.Dense(num_classes)

    def call(self, x):
        x = self.dense1(x)
        return self.dense2(x)

    def model(self):
        x = keras.Input(shape=(28 * 28,))   
        return Model(inputs=[x], outputs=self.call(x))

In [113]:
sub_model= MyModel()

In [115]:
sub_model.model().summary()

Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer_7 (InputLayer)      │ (None, 784)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_16 (Dense)                │ (None, 64)             │        50,240 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_17 (Dense)                │ (None, 10)             │           650 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 50,890 (198.79 KB)

 Trainable params: 50,890 (198.79 KB)

 Non-trainable params: 0 (0.00 B)

In [125]:
sub_model.compile(
    loss=keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    optimizer=keras.optimizers.Adam(learning_rate=0.001),
    metrics=["accuracy"])

In [149]:
sub_model.fit(x_train,y_train,batch_size=32,epochs=5,verbose=1)
sub_model.evaluate(x_test,y_test,batch_size=32,verbose=1)

Epoch 1/5
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 4s 1ms/step - accuracy: 0.8614 - loss: 0.4927
Epoch 2/5
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 2s 1ms/step - accuracy: 0.9549 - loss: 0.1580
Epoch 3/5
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 3s 1ms/step - accuracy: 0.9680 - loss: 0.1110
Epoch 4/5
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 2s 1ms/step - accuracy: 0.9751 - loss: 0.0833
Epoch 5/5
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 2s 1ms/step - accuracy: 0.9799 - loss: 0.0687
313/313 ━━━━━━━━━━━━━━━━━━━━ 0s 922us/step - accuracy: 0.9679 - loss: 0.1059


[0.09010050445795059, 0.9725000262260437]

### Building Dense Layers

In [154]:
class custom_dense(layers.Layer):
    def __init__(self, output_dim, input_dim):
        super().__init__()
        # Initialize weights and biases
        self.w = self.add_weight(
            shape=(input_dim, output_dim),
            initializer="random_normal",
            trainable=True
        )
        self.b = self.add_weight(
            shape=(output_dim,),
            initializer="zeros",
            trainable=True
        )

    def call(self, inputs):
        return tf.matmul(inputs, self.w) + self.b

In [180]:
class MyReLU(keras.layers.Layer):
    def __init__(self):
        super().__init__()

    def call(self, x):
        return tf.math.maximum(x, 0)

In [186]:
class MyModel(keras.Model):
    def __init__(self, num_classes=10):
        super().__init__()
        self.dense1 = custom_dense(64, 784)  # your custom Dense
        self.dense2 = custom_dense(num_classes, 64)
        self.relu = MyReLU() 
    def call(self, x):
        x = self.relu(self.dense1(x))
        return self.dense2(x)

    def model(self):
        x = keras.Input(shape=(28*28,))
        return keras.Model(inputs=[x], outputs=self.call(x))

In [188]:
fin_model=MyModel()
fin_model.model().summary()

Model: "functional_6"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer_17 (InputLayer)     │ (None, 784)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ custom_dense_14 (custom_dense)  │ (None, 64)             │        50,240 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ my_re_lu (MyReLU)               │ (None, 64)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ custom_dense_15 (custom_dense)  │ (None, 10)             │           650 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 50,890 (198.79 KB)

 Trainable params: 50,890 (198.79 KB)

 Non-trainable params: 0 (0.00 B)

In [191]:
fin_model.compile(
    loss=keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    optimizer=keras.optimizers.Adam(learning_rate=0.001),
    metrics=["accuracy"])

In [193]:
fin_model.fit(x_train,y_train,batch_size=32,epochs=5,verbose=1)
fin_model.evaluate(x_test,y_test,batch_size=32,verbose=1)

Epoch 1/5
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 4s 2ms/step - accuracy: 0.8430 - loss: 0.5876
Epoch 2/5
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 4s 2ms/step - accuracy: 0.9498 - loss: 0.1750
Epoch 3/5
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 4s 2ms/step - accuracy: 0.9640 - loss: 0.1226
Epoch 4/5
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 5s 2ms/step - accuracy: 0.9716 - loss: 0.0956
Epoch 5/5
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 5s 2ms/step - accuracy: 0.9750 - loss: 0.0794
313/313 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.9663 - loss: 0.1102


[0.09403443336486816, 0.9718000292778015]